In [1]:
import pandas as pd
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re

In [2]:
def conv_column_arr(data_frame, field, field_arr):
    list_f_arr = []
    dict_hasil = {}
    f_arr = data_frame[field_arr]
    for i in f_arr:
        deleted_simbol = i.replace(" ","").replace("'", "").replace("[","").replace("]","")
        temp = deleted_simbol.split(",")
        
#         dict_hasil[term[i]] = num_doc
        list_f_arr.append(temp)
    dict_hasil = {field: data_frame[field], field_arr:list_f_arr }
    df = pd.DataFrame(dict_hasil)
    
    return df

# Baca File CSV yang Di Perlukan

In [3]:
df_doc_num_token = pd.read_csv("doc_num_token.csv")
df_doc_num_token = conv_column_arr(df_doc_num_token,"doc_num","token")
df_doc_num_token.head()

,doc_num,token
0,1,"[0.39, 1, 750, 1, 780, 1, 850, 1, 870, 1, 875,..."
1,2,"[15:02:20.00, 26-feb-1987, 55, activ, also, am..."
2,3,"[13.5, 15:03:27.51, 26-feb-1987, 31, 7.5, appl..."
3,4,"[-1, -20, 1, 11, 12, 15, 15:07:13.72, 2, 2.70,..."
4,5,"[-1986, 0.99, 1, 1.24, 1.35, 1.56, 1.65, 1.92,..."


In [4]:
df_token_doc_num = pd.read_csv("hasil_preprocessing.csv")
df_token_doc_num = conv_column_arr(df_token_doc_num, "term", "doc_num")
df_token_doc_num.head()

,term,doc_num
0,-1,"[0004, 0008, 0016, 0026, 0059, 0059, 0059, 005..."
1,-10,[0305]
2,-100,[0427]
3,-11,[0116]
4,-110,[0427]


In [5]:
# menghitung jml token
jml_token = 0
for i in range(len(df_doc_num_token)):
    jml_token += len(df_doc_num_token["token"].iloc[i])
jml_token

49088

# Fungsi-fungsi untuk preprocessing query

In [6]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

In [7]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [8]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

In [9]:
stop_words = stopwords.words('english')

In [10]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

In [11]:
def stemming(listOfWords):
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in listOfWords]
    
    return stemmed

### Fungsi utama main preprocessing query

In [12]:
def pre_query(txt):
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = stemming(listOfWords)
    
    return listOfWords

# Fungsi utama menghitung Score dokumen

In [13]:
def hitung_score(list_word,a):
    list_rank = []
    list_doc_num2 = []
    df_hasil = []
    for i in range(len(df_doc_num_token)):
        rank = 1
        doc = df_doc_num_token.iloc[i]
        for j in list_word:
            count = doc["token"].count(j)
            panjang_doc = len(doc["token"])
            p_doc =  (count/panjang_doc) * a
#           ----------------------------------
            temp = df_token_doc_num[df_token_doc_num["term"] == j]
#             print(i, temp)
            count_m = len(temp.iloc[0][1])
            p_m = (count_m/jml_token) * (1-a)
            p = p_doc + p_m
            rank *= p
        list_rank.append(rank)
        list_doc_num2.append(doc["doc_num"])
    dict_hasil = {"doc_num": list_doc_num2, "rank": list_rank}
    df_hasil = pd.DataFrame(dict_hasil)
    df_hasil = df_hasil.sort_values(by=["rank"], ignore_index=True, ascending=False)
    return df_hasil
        

# Coba query

In [14]:
q1 = "1991"
q2 = "Discounts"
q3 = "accessed 9-Mar-87"
q4 = "Analyst appeared"
q5 = "Analyst analyst appeared"
q6 = "Chairman closed the last"

In [15]:
q = pre_query(q2)
search2 = ' '.join(map(str, q))
search2

'discount'

In [16]:
df_hasil = hitung_score(q,0.1)

In [17]:
dict_jawaban = {
    "1991" : [31, 61, 70, 105, 190, 301, 422, 466],
    "discount" : [77, 80, 97, 160, 195, 208, 215, 256, 488],
    "access 9-Mar-87" : [32, 55, 110, 116, 134, 141, 142, 143, 144, 145, 146, 147, 148,
                149, 150, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 261, 
                262, 263, 264, 265, 266, 267, 268, 269, 270, 321, 322, 323, 324, 
                325, 326, 327, 328, 329, 330, 348, 375, 381, 382, 383, 384, 385, 
                386, 387, 388, 389, 390, 441, 442, 443, 444, 445, 446, 447, 448,
                449, 450],
    "analyst appear" : [4, 8, 16, 18, 28, 45, 78, 95, 98, 111, 123, 133, 150, 180, 195, 
                235, 243, 262, 265, 292, 305, 320, 376, 399, 418, 459, 469, 493, 
                495, 496, 497],
    "analyst analyst appear" : [4, 8, 16, 18, 28, 45, 78, 95, 98, 111, 123, 133, 150, 180, 195, 
                235, 243, 262, 265, 292, 305, 320, 376, 399, 418, 459, 469, 493, 
                495, 496, 497],
    "chairman close last" : [1, 12, 18, 26, 28, 40, 43, 47, 52, 60, 61, 69, 78, 95, 98, 102, 
            107, 111, 114, 116, 133, 134, 135, 136, 138, 145, 148, 150, 153, 
            154, 155, 156, 161, 169, 174, 178, 180, 188, 194, 195, 197, 201, 
            203, 204, 207, 208, 214, 216, 224, 227, 230, 231, 232, 233, 234, 
            235, 242, 246, 254, 255, 256, 259, 261, 262, 263, 265, 267, 268, 
            269, 276, 279, 284, 288, 292, 303, 306, 314, 316, 318, 322, 324, 
            328, 330, 338, 339, 340, 343, 350, 355, 367, 371, 372, 375, 376, 
            378, 382, 383, 386, 392, 399, 404, 405, 409, 419, 420, 425, 430, 
            438, 439, 441, 444, 445, 446, 447, 448, 451, 454, 455, 457, 458, 
            463, 484, 488, 491, 492, 494, 495, 496, 498]
}               

In [18]:
print(dict_jawaban)

{'1991': [31, 61, 70, 105, 190, 301, 422, 466], 'discount': [77, 80, 97, 160, 195, 208, 215, 256, 488], 'access 9-Mar-87': [32, 55, 110, 116, 134, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 348, 375, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450], 'analyst appear': [4, 8, 16, 18, 28, 45, 78, 95, 98, 111, 123, 133, 150, 180, 195, 235, 243, 262, 265, 292, 305, 320, 376, 399, 418, 459, 469, 493, 495, 496, 497], 'analyst analyst appear': [4, 8, 16, 18, 28, 45, 78, 95, 98, 111, 123, 133, 150, 180, 195, 235, 243, 262, 265, 292, 305, 320, 376, 399, 418, 459, 469, 493, 495, 496, 497], 'chairman close last': [1, 12, 18, 26, 28, 40, 43, 47, 52, 60, 61, 69, 78, 95, 98, 102, 107, 111, 114, 116, 133, 134, 135, 136, 138, 145, 148, 150, 153, 154, 155, 156, 161, 169, 174, 178, 180, 188, 194, 195

In [19]:
get = dict_jawaban.get(search2)
get

[77, 80, 97, 160, 195, 208, 215, 256, 488]

In [20]:
teratas = df_hasil.head(len(jawaban1)).sort_values(by=["doc_num"], ignore_index=True, ascending=True)


NameError: name 'jawaban1' is not defined

In [21]:
hasil_query = teratas["doc_num"]
salah = []

for i in hasil_query:
    if int(i) not in jawaban1:
        salah.append(i)
        
len(salah)

NameError: name 'teratas' is not defined

In [22]:
teratas

NameError: name 'teratas' is not defined

In [23]:
from flask import Flask, request, render_template, url_for, redirect, jsonify

In [24]:
def search_query(search, res):
    res2 = int(res)
    list_hasil = []
    list_score = []
    q = pre_query(search)
    df_hasil = hitung_score(q,0.1)
    for i in range(0, res2):
        list_hasil.append(df_hasil.iloc[i]["doc_num"])
        list_score.append(df_hasil.iloc[i]["rank"])
    return list_hasil, list_score

In [25]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [26]:
def get_atribut(doc_id):
    int_doc_id = int(doc_id)
    str_doc_id = str(int_doc_id)
    n = 4 - len(str_doc_id)
    doc = ("0"*n) + str_doc_id
    file_path = "DataRouter\\Doc" + doc + ".txt"
    
    txt = readText(file_path)
    
    titlePattern = "<TITLE>(.+)</TITLE>"
    title = re.findall(titlePattern, txt)[0]
    
    datePattern = "<DATE>(.+)</DATE>"
    date = re.findall(datePattern, txt)[0]
    
    bodyPattern = "<BODY>((.+\s+)+)</BODY>"
    body = re.findall(bodyPattern, txt)[0]
    print(body)
    
    return title, date, body

<h1>PUNYA BONTENG TF-IDF</h1>

In [27]:
def searchQuery(_dir, _query):
    file = open(_dir,'r')
    
    listQuery = pre_query(_query)
    
    dictOfQuery = {}
    textPerLine = file.readlines()
    for line in textPerLine:
        temp = line.split('\t')
        if temp[0] in listQuery:
            addLineToDict(dictOfQuery, line)
            
#     for i in dictOfQuery:
#         print(i,' : ',dictOfQuery[i])
#     print('=======================')
    
    file.close()
    
    dictOfRank = computeRank(dictOfQuery, listQuery)
    
    return dictOfRank

In [28]:
def addLineToDict(_dict, _line):
    tempList = _line.split('\t')
    term = tempList[0]
    
    
    termWeigth = tempList[1]
    termWeigth = termWeigth.replace(';\n','')
    termWeigth = termWeigth.split(';')
    listTermTemp = []
    for i in termWeigth:
        i = i.replace('[','')
        i = i.replace(']','')
        i = i.replace(' ','')
        i = i.split(',')
        i = [int(i[0]),float(i[1])]

        listTermTemp.append(i)
        
    _dict[term] = listTermTemp
    
    return _dict

In [29]:
def computeRank(_dictTerm, _query):
    dictQuery = countQuery(_query)
    dictSumOfSim = {}
    dictSumWeigthOfSim = {}
#     print(dictQuery)
    for i in _dictTerm:
        ni = len(_dictTerm[i])
        countWq = countWeigth(ni, dictQuery[i])
#         print(countWq)
        for k in _dictTerm[i]:
            if k[0] in dictSumOfSim:
                dictSumOfSim[k[0]] += k[1]**2 
            else:
                dictSumOfSim[k[0]] = k[1]**2
                
            if k[0] in dictSumWeigthOfSim:
                dictSumWeigthOfSim[k[0]] += countWq * (k[1]**2) 
            else:
                dictSumWeigthOfSim[k[0]] = countWq * (k[1]**2)
                
#     print(dictSumOfSim)
#     print(dictSumWeigthOfSim)
                
    dictSqrtOfSim = findSqrt(dictSumOfSim)
    
    dictOfRank = findRank(dictSqrtOfSim, dictSumWeigthOfSim)
    
    return dictOfRank

In [30]:
def countQuery(_query):
    dictOfQuery = {}
    for i in _query:
        if i in dictOfQuery:
            dictOfQuery[i] += 1
        else:
            dictOfQuery[i] = 1
            
    return dictOfQuery

In [31]:
def findRank(_dictSqrt, _dictWeigth):
#     print('sqrt : ',_dictSqrt)
#     print('weigth : ',_dictWeigth)
    dictOfRank = {}
    for i in _dictSqrt:
        dictOfRank[i] = (_dictWeigth[i]/_dictSqrt[i])
        
    return({k: v for k, v in sorted(dictOfRank.items(), key=lambda item: item[1], reverse=True)})

In [32]:
import math

def findSqrt(_dict):
    for i in _dict:
        _dict[i] = math.sqrt(_dict[i])
    
    return _dict

In [33]:
def countWeigth(_ni, _fij):
    
    file = open('sum_of_file.txt','r')
    sumOfFile = int(file.read())
    file.close()
    
    tf = computeTFLog(_fij)
    idf = countIDF(sumOfFile, _ni)
    weigth = tf * idf
    weigth = weigth**2
    
    return weigth

In [34]:
import math
#tf = 1 + log(i,j)
def computeTFLog(_fi):
    return 1 + math.log(_fi,2)

In [35]:
import math
#idf = log(N/ni)
def countIDF(_N, _ni):
#     print('N : ',_N)
#     print('_ni : ',_ni)
    return math.log(1+(_N/_ni),2)

In [36]:
import time
start_time = time.time()
rank = searchQuery('tf-idf.txt', 'Discount')
print("--- %s seconds ---" % (time.time() - start_time))
print('Rank : ', rank)

--- 0.023421764373779297 seconds ---
Rank :  {97: 510.75683569545686, 80: 395.175431406038, 77: 197.587715703019, 160: 197.587715703019, 195: 197.587715703019, 208: 197.587715703019, 215: 197.587715703019, 256: 197.587715703019, 488: 33.92396659928883}


In [37]:
def to_list(rank, res):
    res2 = int(res)
    doc = []
    score = []
    doc2 = []
    score2 = []
    for x in rank:
        doc.append(x)
        score.append(rank[x])
    if len(doc) > res2:
        for i in range(0, res2):
            doc2.append(doc[i])
            score2.append(score[i])
        return doc2, score2
    else:
        return doc, score

In [38]:
def calRecall(hasil, kunci):
    list_match = []
    for i in range(0,len(kunci)):
        for j in range(0,len(hasil)):
            if kunci[i] == hasil[j]:
                list_match.append(hasil[j])

    recall = 100 * (len(list_match)/len(kunci))
    return recall            

In [39]:
def calPrecision(hasil, kunci):
    list_match = []
    for i in range(0,len(kunci)):
        for j in range(0,len(hasil)):
            if kunci[i] == hasil[j]:
                list_match.append(hasil[j])

    precision = 100 * (len(list_match)/len(hasil))
    return precision

<h1>================</h1>

In [40]:
ui = Flask(__name__)
x = []
@ui.route('/')
def index():
    return render_template('base.html')


@ui.route('/search', methods=['POST', 'GET'])
def search():
    if request.method == 'POST':
        list_title = []
        list_date = []
        list_body =[]
        search = request.form["search"]
        method = request.form.get('met')
        result = request.form.get('res')
        if method == "1":
            start_time = time.time()
            hasil, score = search_query(search, result)
        else :
            start_time = time.time()
            rank = searchQuery('tf-idf.txt', search)
            hasil, score = to_list(rank, result)
        for i in hasil:
            title, date, body = get_atribut(i)
            list_title.append(title)
            list_date.append(date)
            list_body.append(body)
        search = pre_query(search)
        search2 = ' '.join(map(str, search))
        rec = calRecall(hasil, dict_jawaban.get(search2))
        prec = calPrecision(hasil, dict_jawaban.get(search2))
        ptime = (time.time() - start_time)
        return render_template('respond.html', title = list_title, date = list_date,body=list_body, score = score, leng = len(list_title), time = ptime, recal = rec, precision = prec)
    else:
        return render_template('index.html')
    
if __name__ == "__main__":
    ui.run()

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2020 20:03:40] "POST /search HTTP/1.1" 200 -
("The House Ways and Means Committee\nis moving toward passage of a trade bill that sponsors said was\nintended to help open foreign markets to U.S. agricultural\ngoods and to modify some U.S. agricultural trade laws.\n    The trade subcommittee voted to require President Reagan to\ntake into account the potential harm to U.S. agricultural\nexports of any trade retaliation he might impose for foreign\nunfair trade practices against other domestic industries.\n    The bill would allow U.S. agricultural producers to seek\ngovernment monitoring of imports if there is a reasonable\nchance the industry would be harmed by an import surge.\n    The full Ways and Means Committee is to consider the bill\nnext week and congressional sources